## Veritabanı Bağlantısı
### DBAPI
* Python'da RDBMS'lere bağlanmak için kullanılan bir çok kütüphane mevcuttur.
  * Hatta mesela MSSQL için birden fazla kütüphane bulabilirsiniz.
* Veritabanlarına bağlanan kütüphanler DBAPI kütüphaneleridir. Temelde bu kütüphaneler aynı API'yi kullanır.
  * Bu sayade farklı DB'lerde hemen hemen aynı şekilde çalışan kodlar yazabiliriz.
* Not:
  * ilişkisel veritabanları ile çalışırken:
    * SQL dilini kullanırız. Bu dil ANSI/ISO standartlarıyla korunmaktadır.
    * Vendorler bu dilde olmayan özellikleri dili biraz değiştirerek ekleyebilirler.
      * MSSQL (T-SQL), Oracle (PL/SQL), PostgreSQL (PL/pgSQL) gibi.
  * Bağlanırken bağlantı cümlesine ihtiyaç var. Servera ve kullanılan providera göre bağlantı cümlesi değişebilir
    * Tüm bağlantı cümlelerine https://www.connectionstrings.com/ adresinden erişebilirsiniz.

#### SQLite
* sqlite ayrıca bir service kurmadan kullanılabilecek light RDBMS ürünüdür. MSSQL, Oracle, MySQL gibi bir service kurmanıza gerek yoktur.
  * sqlite https://www.sqlite.org/download.html adresinden indirilebilir.
  * IDE https://sqlitestudio.pl/ adresinden indirilebilir.

In [1]:
import sqlite3
#sqlitee bağlanmak için kullanılabilecek bir modul/pakettir. DBPI'den türetilmiştir.

In [2]:
import os
os.makedirs("./databases", exist_ok=True)

In [5]:
#Data Source=c:\mydb.db;Version=3;
#Data Source=:memory:;Version=3;New=True;
con = sqlite3.connect("./databases/mesajDB.db") #bağlantı cümleisni belirtip bağlanıyoruz.

#dir(con)

In [6]:
#sql sorgusu çalıştırmamızı sağlar. Bir cursor nesnesi döner.
con.execute("""
CREATE TABLE IF NOT EXISTS mesaj (
    id int,
    mesaj nvarchar(100)
)
""")

In [ ]:
#tabloyu silmek için
#con.execute("DROP TABLE IF EXISTS mesaj")

In [7]:
#satır insert edelim
con.execute("INSERT INTO mesaj (id, mesaj) VALUES (1, '1. Kayıt')")
con.execute("INSERT INTO mesaj (id, mesaj) VALUES (2, '2. Kayıt')")
con.execute("INSERT INTO mesaj (id, mesaj) VALUES (3, '3. Kayıt')")
con.execute("INSERT INTO mesaj (id, mesaj) VALUES (4, '4. Kayıt')")
con.execute("INSERT INTO mesaj (id, mesaj) VALUES (5, '5. Kayıt')")

#veritabanlarında auto commit özelilği olmayabilir. Açık olmayaabilir.
con.commit() #değişiklikleri kalıcı hale getirir. commit edilmezse oturum kapatıldığında rollback yapılır.
#con.rollback() #yapılan işlemleri geri alır.

In [8]:
#kayıtları okuyalım
'''
imlec = con.cursor() #önce imlec nesnesi oluşsun 
imlec.execute("SELECT * FROM mesaj") #aynı imlec ile farklı sorgular çalıştırılabilir.
'''

imlec = con.execute("SELECT * FROM mesaj LIMIT 2") #üstten 2 satır alsın.

In [9]:
#imlec hakknda bilgi alalım
imlec.description #alanlar hakkında bilgi verir.

(('id', None, None, None, None, None, None),
 ('mesaj', None, None, None, None, None, None))

In [10]:
#kolon isimlerii alalım.
kolonAd = [kolon[0] for kolon in imlec.description]
print(kolonAd)

['id', 'mesaj']


In [11]:
#satıları çekelim.
satirlar = imlec.fetchall() #tüm satırları alır.
satirlar

[(1, '1. Kayıt'), (2, '2. Kayıt')]

In [19]:
#ekrana yazdıralım
fmt="|{:^5}|{:^20}|"
print(fmt.format(*kolonAd))
print("-"*27)

for satir in satirlar:
    print(fmt.format(*satir))

| id  |       mesaj        |
---------------------------
|  1  |      1. Kayıt      |
|  2  |      2. Kayıt      |


### Pandas ile Bağlanmak

In [20]:
import sqlite3

con = sqlite3.connect("./databases/mesajDB.db")

In [21]:
import pandas as pd

df = pd.read_sql("SELECT * FROM mesaj", con)  #sqlite connectionı verdik.

type(df)

pandas.core.frame.DataFrame

In [26]:
df #tüm satırlar
df.head(2) #üstten istenilen satır
df.tail(2) #alttan istenilen satır

df.sample(2) #rastgele satır

#
df.shape #5 satır 2 sutun

(5, 2)

In [36]:
con.close() #bağlantıyı kapatır.

#### MSSQL (veya Oracle) ile Çalışmak
* MSSQL'a bağlanmak için `pyodbc` ve `pymssql`  kütüphanalerini kullanabiliriz.
  * `pyodbc` MSSQL ve Oracle'a bağlanmak için kullanılabilir.
  * `pymssql` sadece MSSQL için kullanılabilir.
* MSSQL'de 
  * `pip install pyodbc` veya `pip install pymssql` ile kütüphaneleri yükleyin.
  * `import pyodbc` veya `import pymssql` ile kütüphaneleri projenize dahil edin.
  * `pyodbc.connect("baglantiCumlesi")` Driver bilsini verip bağlantı cümlesi oluşturacağız.
* Oracle'da
  * `pip install cx_Oracle` ile kütüphaneyi yükleyin.
  * `import cx_Oracle` ile kütüphaneyi projenize dahil edin.
  * `cx_Oracle.connect("baglantiCumlesi")` ile bağlantı cümlesi oluşturacağız.

In [38]:
import pyodbc

In [39]:
pyodbc.drivers() #kurulu driverları listeler.

['SQL Server',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)',
 'Oracle in OraDB19Home1',
 'ODBC Driver 17 for SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)']

In [40]:
#Driver={ODBC Driver 17 for SQL Server};Server=myServerAddress;Database=myDataBase;UID=myUsername;PWD=myPassword;
#Driver={ODBC Driver 17 for SQL Server};Server=myServerAddress;Database=myDataBase;Trusted_Connection=yes;

cnf = {
        "driver": "ODBC Driver 17 for SQL Server",
        "server": "localhost",
        "database": "master",
        "user": "sa",
        "password": "123"
}

con_str="Driver={{{driver}}};Server={server};Database={database};UID={user};PWD={password};".format_map(cnf)

print(con_str)

#baglantı kuralım.
con = pyodbc.connect(con_str)

Driver={ODBC Driver 17 for SQL Server};Server=localhost;Database=master;UID=sa;PWD=123;


In [41]:
imlec = con.execute("SELECT @@version")

In [42]:
versiyon = imlec.fetchone()
versiyon

('Microsoft SQL Server 2019 (RTM-CU25-GDR) (KB5036335) - 15.0.4360.2 (X64) \n\tMar 19 2024 00:23:01 \n\tCopyright (C) 2019 Microsoft Corporation\n\tEnterprise Edition (64-bit) on Windows 10 Pro 10.0 <X64> (Build 19045: ) (Hypervisor)\n',)

In [43]:
print(versiyon[0])

Microsoft SQL Server 2019 (RTM-CU25-GDR) (KB5036335) - 15.0.4360.2 (X64) 
	Mar 19 2024 00:23:01 
	Copyright (C) 2019 Microsoft Corporation
	Enterprise Edition (64-bit) on Windows 10 Pro 10.0 <X64> (Build 19045: ) (Hypervisor)



In [44]:
con.close() #bağlantıyı kapatır.